In [9]:
from numpy import *
from itertools import combinations

In [10]:
dataSet = [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [11]:
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
                
    C1.sort()
    return list(map(frozenset, C1))#use frozen set so we can use it as a key in a dict    

In [12]:
def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if not can in ssCnt: ssCnt[can]=1
                else: ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData

In [13]:
def checkSubsetFrequency(candidate, Lk, k):
    if k>1:    
        subsets = list(combinations(candidate, k))
    else:
        return True
    for elem in subsets:
        if not frozenset(elem) in Lk:
            return False
    return True

In [14]:
def aprioriGen(Lk, k): #creates Ck
    resList = [] #result set
    candidatesK = [] 
    lk = sorted(set([item for t in Lk for item in t])) #get and sort elements from frozenset
    candidatesK = list(map(frozenset, combinations(lk, k)))
    for can in candidatesK:
        if checkSubsetFrequency(can, Lk, k-1):
            resList.append(can)
    return resList

In [15]:
def apriori(dataSet, minSupport):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#scan DB to get Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    #remove empty last itemset from L
    if L[-1] == []:
        L.pop()
    return L, supportData

In [28]:
L, suppData = apriori(dataSet, 0.5)

In [29]:
L

[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})],
 [frozenset({3, 5}), frozenset({2, 5}), frozenset({2, 3}), frozenset({1, 3})],
 [frozenset({2, 3, 5})]]

In [32]:
def generateRules(L, supportData, minConf):  #supportData is a dict coming from scanD
    for i in range(1, len(L)): #only get the sets with two or more items
        for item in L[i]: #for each item in a level
            for j in range(1, i+1): # i+1 equal to length of an item
                lhsList = list(map(frozenset, combinations(item, j)))
                for lhs in lhsList:
                    rhs = item.difference(lhs)
                    conf = supportData[item]/supportData[rhs]
                    if conf >= minConf:
                        print(list(lhs), " ==> ", list(rhs), " [", supportData[item], ", ", conf,"]", sep="")

In [33]:
generateRules(L, suppData, 0.5)

[3] ==> [5] [0.5, 0.6666666666666666]
[5] ==> [3] [0.5, 0.6666666666666666]
[2] ==> [5] [0.75, 1.0]
[5] ==> [2] [0.75, 1.0]
[2] ==> [3] [0.5, 0.6666666666666666]
[3] ==> [2] [0.5, 0.6666666666666666]
[1] ==> [3] [0.5, 0.6666666666666666]
[3] ==> [1] [0.5, 1.0]
[2] ==> [3, 5] [0.5, 1.0]
[3] ==> [2, 5] [0.5, 0.6666666666666666]
[5] ==> [2, 3] [0.5, 1.0]
[2, 3] ==> [5] [0.5, 0.6666666666666666]
[2, 5] ==> [3] [0.5, 0.6666666666666666]
[3, 5] ==> [2] [0.5, 0.6666666666666666]
